In [ ]:
!pip install ultralytics

"""
Notebook 2: Model Training
Construction Safety Detection dengan YOLOv8
"""

from ultralytics import YOLO
import torch
import yaml
from pathlib import Path


# 1. CHECK GPU AVAILABILITY

In [ ]:
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    device = 'cuda'
else:
    print("Using CPU")
    device = 'cpu'


CUDA Available: True
GPU: Tesla T4



# 2. LOAD CONFIGURATION

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/MyDrive/Colab Notebooks/Capstone 4/extracted/data.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("\nDataset Configuration:")
print(f"Classes: {config['names']}")
print(f"Number of classes: {config['nc']}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Dataset Configuration:
Classes: ['helmet', 'no-helmet', 'no-vest', 'person', 'vest']
Number of classes: 5



# 3. LOAD PRE-TRAINED MODEL

In [ ]:

model = YOLO('yolov8s.pt')  # Menggunakan Small model
print("\n✅ YOLOv8 Small model loaded")


✅ YOLOv8 Small model loaded



# 4. TRAINING CONFIGURATION

In [ ]:


# ============================================
training_config = {
    'data': '/content/drive/MyDrive/Colab Notebooks/Capstone 4/extracted/data.yaml',
    'epochs': 100,              # Jumlah epoch training
    'imgsz': 640,               # Ukuran input image
    'batch': 16,                # Batch size
    'patience': 20,             # Early stopping patience
    'save': True,               # Save checkpoint
    'device': device,           # cuda or cpu
    'workers': 4,               # Number of workers
    'project': '/content/drive/MyDrive/Colab Notebooks/Capstone 4/models',     # Project directory
    'name': 'construction_safety',  # Experiment name
    'exist_ok': True,           # Overwrite existing

    # Optimization
    'optimizer': 'AdamW',       # Optimizer
    'lr0': 0.01,                # Initial learning rate
    'lrf': 0.01,                # Final learning rate (lr0 * lrf)
    'momentum': 0.937,          # SGD momentum
    'weight_decay': 0.0005,     # Weight decay

    # Augmentation (tanpa mengubah geometris)
    'hsv_h': 0.015,             # Hue augmentation
    'hsv_s': 0.7,               # Saturation augmentation
    'hsv_v': 0.4,               # Value (brightness) augmentation
    'degrees': 0.0,             # Rotation (set 0 - tidak rotate)
    'translate': 0.0,           # Translation (set 0)
    'scale': 0.0,               # Scale (set 0)
    'shear': 0.0,               # Shear (set 0)
    'flipud': 0.0,              # Flip up-down probability (set 0)
    'fliplr': 0.0,              # Flip left-right probability (set 0)
    'mosaic': 0.0,              # Mosaic augmentation (set 0)

    # Validation
    'val': True,                # Validate during training
    'plots': True,              # Save training plots
}

print("\nTraining Configuration:")
for key, value in training_config.items():
    print(f"  {key}: {value}")


Training Configuration:
  data: /content/drive/MyDrive/Colab Notebooks/Capstone 4/extracted/data.yaml
  epochs: 100
  imgsz: 640
  batch: 16
  patience: 20
  save: True
  device: cuda
  workers: 4
  project: /content/drive/MyDrive/Colab Notebooks/Capstone 4/models
  name: construction_safety
  exist_ok: True
  optimizer: AdamW
  lr0: 0.01
  lrf: 0.01
  momentum: 0.937
  weight_decay: 0.0005
  hsv_h: 0.015
  hsv_s: 0.7
  hsv_v: 0.4
  degrees: 0.0
  translate: 0.0
  scale: 0.0
  shear: 0.0
  flipud: 0.0
  fliplr: 0.0
  mosaic: 0.0
  val: True
  plots: True



# 5. START TRAINING

In [ ]:


print("\n" + "="*60)
print("🚀 STARTING TRAINING...")
print("="*60)

results = model.train(**training_config)

print("\n✅ Training Completed!")
print(f"Best model saved at: /content/drive/MyDrive/Colab Notebooks/Capstone 4/models/construction_safety/weights/best.pt")


🚀 STARTING TRAINING...
Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Colab Notebooks/Capstone 4/extracted/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=construction_safety, nbs=64, nms=False, opset=None, optimize=False, optimize


# 6. EVALUATE ON VALIDATION SET

In [ ]:
print("\n" + "="*60)
print("📊 EVALUATING MODEL ON VALIDATION SET...")
print("="*60)

metrics = model.val()

print("\nValidation Metrics:")
print(f"  mAP50: {metrics.box.map50:.4f}")
print(f"  mAP50-95: {metrics.box.map:.4f}")
print(f"  Precision: {metrics.box.mp:.4f}")
print(f"  Recall: {metrics.box.mr:.4f}")

# Per-class metrics
print("\nPer-Class Metrics:")
for i, class_name in enumerate(config['names']):
    print(f"  {class_name}:")
    print(f"    Precision: {metrics.box.class_result(i)[0]:.4f}")
    print(f"    Recall: {metrics.box.class_result(i)[1]:.4f}")
    print(f"    mAP50: {metrics.box.class_result(i)[2]:.4f}")



📊 EVALUATING MODEL ON VALIDATION SET...
Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,127,519 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 1.0±0.7 ms, read: 43.9±44.2 MB/s, size: 918.6 KB)
val: Scanning /content/drive/MyDrive/Colab Notebooks/Capstone 4/extracted/valid/labels.cache... 119 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 119/119 144.3Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 1.8it/s 4.4s
                   all        119        715      0.749      0.589      0.689      0.402
                helmet        117        232      0.894      0.802      0.886      0.524
             no-helmet          6         11      0.421      0.273      0.314      0.191
               no-vest         52         90      0.748      0.444      0.638       0.34
                person        115        241      0.899


# 7. TEST ON TEST SET (OPTIONAL)

In [ ]:

print("\n" + "="*60)
print("🧪 TESTING ON TEST SET...")
print("="*60)

# Load best model
best_model = YOLO('/content/drive/MyDrive/Colab Notebooks/Capstone 4/models/construction_safety/weights/best.pt')

# Test
test_results = best_model.val(data='/content/drive/MyDrive/Colab Notebooks/Capstone 4/extracted/data.yaml', split='test')

print("\nTest Set Metrics:")
print(f"  mAP50: {test_results.box.map50:.4f}")
print(f"  mAP50-95: {test_results.box.map:.4f}")
print(f"  Precision: {test_results.box.mp:.4f}")
print(f"  Recall: {test_results.box.mr:.4f}")



🧪 TESTING ON TEST SET...
Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,127,519 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.5±0.2 ms, read: 0.1±0.1 MB/s, size: 87.5 KB)
val: Scanning /content/drive/MyDrive/Colab Notebooks/Capstone 4/extracted/test/labels... 90 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 90/90 0.7it/s 2:04
val: New cache created: /content/drive/MyDrive/Colab Notebooks/Capstone 4/extracted/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 1.6it/s 3.9s
                   all         90        623      0.713      0.598      0.634      0.339
                helmet         82        195      0.835      0.815      0.851      0.456
             no-helmet         11         24      0.536      0.208       0.22     0.0706
               no-vest         31         61      0.625      0.557      0


# 8. SAVE TRAINING SUMMARY

In [ ]:

import json

summary = {
    'model': 'YOLOv8s',
    'dataset': 'Construction Safety',
    'num_classes': config['nc'],
    'classes': config['names'],
    'epochs': training_config['epochs'],
    'validation_metrics': {
        'mAP50': float(metrics.box.map50),
        'mAP50-95': float(metrics.box.map),
        'precision': float(metrics.box.mp),
        'recall': float(metrics.box.mr)
    },
    'test_metrics': {
        'mAP50': float(test_results.box.map50),
        'mAP50-95': float(test_results.box.map),
        'precision': float(test_results.box.mp),
        'recall': float(test_results.box.mr)
    }
}

with open('/content/drive/MyDrive/Colab Notebooks/Capstone 4/models/training_summary.json', 'w') as f:
    json.dump(summary, f, indent=4)

print("\n✅ Training summary saved to: /content/drive/MyDrive/Colab Notebooks/Capstone 4/models/training_summary.json")
print("\n🎉 All Done! Check the 'models/construction_safety' folder for results.")



✅ Training summary saved to: /content/drive/MyDrive/Colab Notebooks/Capstone 4/models/training_summary.json

🎉 All Done! Check the 'models/construction_safety' folder for results.
